# Neo4J v058 test1 - Trellis : Job based Analysis
================

## Set up the environment


### Install py2neo for querying Neo4J 

In [340]:
#!pip3 install -U py2neo

# add python path of py2neo in system

#!pip3 install -U neotime
#!pip3 install -U neobolt
#!pip3 install -U pandas-gbq

### Import Packages

In [341]:
from py2neo import Graph
from google.cloud import storage
import yaml

import pandas as pd
import pandas_gbq

import numpy as np
import subprocess
import matplotlib 
import matplotlib.pyplot as plt
import seaborn as sns

#plt.style.use('fivethirtyeight')
#plt.style.use('ggplot')
pd.set_option('display.float_format', lambda x: '%.2f' % x)

### Load Neo4J DB

In [342]:
## Option 1 : Read DB and Account Information in Google Storage (YAML)

# create storage client
storage_client = storage.Client()
# get bucket with name
bucket = storage_client.get_bucket('gbsc-gcp-project-mvp-test-trellis')
# get bucket data as blob
blob = bucket.get_blob('credentials/test-wgs35.yaml')
# convert to string
yaml_data = blob.download_as_string()

account = yaml.load(yaml_data, Loader=yaml.FullLoader)

## Main Account
graph = Graph(account['NEO4J_SCHEME']+'://'+account['NEO4J_HOST']+":"+str(account['NEO4J_PORT']), auth=(account['NEO4J_USER'],account['NEO4J_PASSPHRASE']))

-------
## FQ2U Job

### FQ2U table

In [343]:
## Query
query = "Match (fu:Job:FastqToUbam)-[:STATUS]->(s:Dstat) RETURN fu.sample AS sample, fu.readGroup AS fq2urg_gatkid, fu.duplicate AS dup, fu.machineType AS vm_type, fu.durationMinutes as job_runtime, s.status as dstat_status, s.statusMessage as dstat_msg, s.logging as dstat_log"
job_fq2u = graph.run(query).to_data_frame()
job_fq2u.set_index('sample')

## Variable
num_fq2u_sample=len(job_fq2u['sample'].unique())
num_fq2u_job=len(job_fq2u)

## Print (Info)
print("The number of samples with FQ2U jobs : " + str(num_fq2u_sample))
print("The number of FQ2U jobs : " + str(num_fq2u_job))

## Bigquery Table Format
job_fq2u['job_group']='GATK'
job_fq2u['vm_exp_cnt']=1
job_fq2u['job']='FQ2U'
job_fq2u['vm_cnt']=1
job_fq2u['vm_disk']=None
job_fq2u['vm_avg_runtime']=job_fq2u['job_runtime']
columnlist=['sample','job_group','job','fq2urg_gatkid','dup','vm_exp_cnt','vm_cnt','vm_avg_runtime','job_runtime','vm_type','vm_disk','dstat_status','dstat_msg','dstat_log']
job_fq2u=job_fq2u[columnlist]

#display(job_fq2u.head())

The number of samples with FQ2U jobs : 288
The number of FQ2U jobs : 1156


### FQ2U Duplication Check

In [344]:
## Query
fq2u_dup=job_fq2u.loc[job_fq2u['dup']==True,:]

## Variable
num_dup_fq2u_sample=len(fq2u_dup['sample'].unique())
num_dup_fq2u_job=len(fq2u_dup)

print("The number(percentage) of samples with duplicated FQ2U jobs : " + str(len(fq2u_dup['sample'].unique()))+" ("+'{:2f}'.format((num_dup_fq2u_sample/num_fq2u_sample)*100)+"%)")
print("The number(percentage) of FQ2U duplicated jobs : " + str(num_dup_fq2u_job)+" ("+'{:2f}'.format((num_dup_fq2u_job/num_fq2u_job)*100)+"%)")

#display(test)

The number(percentage) of samples with duplicated FQ2U jobs : 0 (0.000000%)
The number(percentage) of FQ2U duplicated jobs : 0 (0.000000%)


-------
## GATK Job

### GATK table

In [345]:
## Query
query = "MATCH (j:Job:CromwellWorkflow)-[:STATUS]->(s:Dstat) RETURN j.sample AS sample, j.cromwellWorkflowId AS fq2urg_gatkid, \
j.duplicate AS dup, j.durationMinutes as job_runtime, j.machineType as vm_type, s.status as dstat_status, s.statusMessage as dstat_msg, s.logging as dstat_log"
job_gatk = graph.run(query).to_data_frame()
job_gatk.set_index('sample')

## Variable
num_gatk_sample=len(job_gatk['sample'].unique())
num_gatk_job=len(job_gatk)

## Print (Info)
print("The number of samples with GATK jobs : " + str(num_gatk_sample))
print("The number of GATK jobs : " + str(num_gatk_job))

## Bigquery Table Format
job_gatk['job_group']='GATK'
job_gatk['vm_exp_cnt']=1
job_gatk['job']='cromwell'
job_gatk['vm_cnt']=1
job_gatk['vm_avg_runtime']=job_gatk['job_runtime']
job_gatk['vm_disk']=None
columnlist=['sample','job_group','job','fq2urg_gatkid','dup','vm_exp_cnt','vm_cnt','vm_avg_runtime','job_runtime','vm_type','vm_disk','dstat_status','dstat_msg','dstat_log']
job_gatk=job_gatk[columnlist]

display(job_gatk.head())

The number of samples with GATK jobs : 287
The number of GATK jobs : 287


,sample,job_group,job,fq2urg_gatkid,dup,vm_exp_cnt,vm_cnt,vm_avg_runtime,job_runtime,vm_type,vm_disk,dstat_status,dstat_msg,dstat_log
0,SHIP4948531,GATK,cromwell,e2bbce13-cc8d-4c53-bb02-93d52abb4785,None,1,1,1278,1278,custom-2-12288,None,SUCCESS,Success,gs://gbsc-gcp-project-mvp-test-from-personalis...
1,SHIP5119492,GATK,cromwell,cecec895-9d04-4be2-92a7-f3fecc9a25ce,None,1,1,1208,1208,custom-2-12288,None,SUCCESS,Success,gs://gbsc-gcp-project-mvp-test-from-personalis...
2,SHIP5119501,GATK,cromwell,e577868e-b8e7-4c26-9631-3e6790e88255,None,1,1,1900,1900,custom-2-12288,None,SUCCESS,Success,gs://gbsc-gcp-project-mvp-test-from-personalis...
3,SHIP5125869,GATK,cromwell,62f50ed3-29f4-4f6d-8293-eca7fb1535e1,None,1,1,1374,1374,custom-2-12288,None,SUCCESS,Success,gs://gbsc-gcp-project-mvp-test-from-personalis...
4,SHIP4946371,GATK,cromwell,eff32b27-b937-4854-8e31-e1f46da24ff0,None,1,1,1232,1232,custom-2-12288,None,SUCCESS,Success,gs://gbsc-gcp-project-mvp-test-from-personalis...


### GATK Duplication Check

In [346]:
## Query
gatk_dup=job_gatk.loc[job_gatk['dup']==True,:]

## Variable
num_dup_gatk_sample=len(gatk_dup['sample'].unique())
num_dup_gatk_job=len(gatk_dup)

print("The number(percentage) of samples with duplicated GATK jobs : " + str(len(gatk_dup['sample'].unique()))+" ("+'{:2f}'.format((num_dup_gatk_sample/num_gatk_sample)*100)+"%)")
print("The number(percentage) of GATK duplicated jobs : " + str(num_dup_gatk_job)+" ("+'{:2f}'.format((num_dup_gatk_job/num_gatk_job)*100)+"%)")

#display(test)

The number(percentage) of samples with duplicated GATK jobs : 0 (0.000000%)
The number(percentage) of GATK duplicated jobs : 0 (0.000000%)


### GATK vm_exp_cnt and add_vm

In [347]:
## expected vm data frame
#vm_exp_cnt_df=pd.read_excel("./GATKstep_expected_vm.xlsx")

In [348]:
#merged_vm_cnt_df=pd.merge(vm_exp_cnt_df,job_gatk,left_on=['job'],right_on=['job'],how='right')
#merged_attemps_df['added_vm']=job_gatk['vm_cnt']-merged_attemps_df['vm_exp_cnt']

-------
## GATK substeps

### vm_cnt table

In [349]:
## Query
query = "MATCH (g:Job:CromwellWorkflow)-[:LED_TO*]->(s:CromwellStep)-[:HAS_ATTEMPT]-()-[*0..100]->(j:Job) \
WHERE g.cromwellWorkflowId=s.cromwellWorkflowId RETURN g.sample as sample, s.cromwellWorkflowId as fq2urg_gatkid, \
s.wdlCallAlias as job, count(distinct j) as vm_cnt, (max(j.stopTimeEpoch)-min(j.startTimeEpoch))/60 as job_runtime, avg(j.durationMinutes) as vm_avg_runtime, j.machineType as vm_type"
#query = "MATCH (j:Job:CromwellWorkflow)-[:STATUS]->(s:Dstat) RETURN j.sample AS sample, j.duplicate AS dup, j.durationMinutes as job_runtime, s.status as dstat_status, s.statusMessage as dstat_msg, s.logging as dstat_log"
job_gatk_step = graph.run(query).to_data_frame()
job_gatk_step.set_index('sample')

## Variable
num_gatk_sample=len(job_gatk_step['sample'].unique())
num_gatk_subjobs=len(job_gatk_step)

## Print (Info)
print("The number of samples with GATK steps : " + str(num_gatk_sample))
print("The number of GATK subjobs : " + str(num_gatk_subjobs))

The number of samples with GATK steps : 287
The number of GATK subjobs : 4603


### GATK Duplication Check

In [350]:
## Bigquery Table Format
job_gatk_step['job_group']='GATK'
job_gatk_step['vm_disk']=None

job_gatk_info=job_gatk[['sample','fq2urg_gatkid','dup','dstat_status','dstat_msg','dstat_log']]
job_gatk_stepm=pd.merge(job_gatk_info, job_gatk_step, left_on=['sample','fq2urg_gatkid'], right_on=['sample','fq2urg_gatkid'], how='right')

display(job_gatk_stepm.head())

,sample,fq2urg_gatkid,dup,dstat_status,dstat_msg,dstat_log,job,job_runtime,vm_avg_runtime,vm_cnt,vm_type,job_group,vm_disk
0,SHIP4948531,e2bbce13-cc8d-4c53-bb02-93d52abb4785,None,SUCCESS,Success,gs://gbsc-gcp-project-mvp-test-from-personalis...,createsequencegroupingtsv,3.31,3.00,1,custom-1-2048,GATK,None
1,SHIP4948531,e2bbce13-cc8d-4c53-bb02-93d52abb4785,None,SUCCESS,Success,gs://gbsc-gcp-project-mvp-test-from-personalis...,scatterintervallist,3.68,3.00,1,custom-1-2048,GATK,None
2,SHIP4948531,e2bbce13-cc8d-4c53-bb02-93d52abb4785,None,SUCCESS,Success,gs://gbsc-gcp-project-mvp-test-from-personalis...,samtofastqandbwamemandmba,166.64,117.57,5,custom-16-14848,GATK,None
3,SHIP4948531,e2bbce13-cc8d-4c53-bb02-93d52abb4785,None,SUCCESS,Success,gs://gbsc-gcp-project-mvp-test-from-personalis...,sumfloats,3.03,3.00,1,custom-2-10240,GATK,None
4,SHIP4948531,e2bbce13-cc8d-4c53-bb02-93d52abb4785,None,SUCCESS,Success,gs://gbsc-gcp-project-mvp-test-from-personalis...,markduplicates,322.32,322.00,1,custom-2-7168,GATK,None


### GATK vm_exp_cnt and add_vm

In [351]:
## expected vm data frame
vm_exp_cnt_df=pd.read_excel("./GATKstep_expected_vm.xlsx")

In [352]:
job_gatk_stepm=pd.merge(vm_exp_cnt_df,job_gatk_stepm,left_on=['job'],right_on=['job'],how='right')
columnlist=['sample','job_group','job','fq2urg_gatkid','dup','vm_exp_cnt','vm_cnt','vm_avg_runtime','job_runtime','vm_type','vm_disk','dstat_status','dstat_msg','dstat_log']
job_gatk_stepm=job_gatk_stepm[columnlist]
#merged_attemps_df['added_vm']=job_gatk['vm_cnt']-merged_attemps_df['vm_exp_cnt']

-------
## QC Job

### Fastqc

In [353]:
query="MATCH (j:Job:BamFastqc)-[:STATUS]->(d:Dstat) RETURN j.sample as sample, j.name as job, j.duplicate as dup, j.machineType as vm_type, j.durationMinutes as job_runtime, j.diskSize as vm_disk, d.status as dstat_status, d.statusMessage as dstat_msg, d.logging as dstat_log"
job_fastqc = graph.run(query).to_data_frame()

job_fastqc['job_group']='QC'
job_fastqc['fq2urg_gatkid']=None
job_fastqc['vm_exp_cnt']=1
job_fastqc['vm_cnt']=1
job_fastqc['vm_avg_runtime']=job_fastqc['job_runtime']

job_fastqc=job_fastqc[columnlist]
job_fastqc.set_index('sample')

## duplication check

print('The number of rows : ' + str(len(job_fastqc)))
print('The number of duplicated jobs : ' + str(len(job_fastqc.loc[job_fastqc['dup']==True,:])))

job_fastqc.drop_duplicates(columnlist,keep='first',inplace=True)

print('The number of rows dropped duplications : ' + str(len(job_fastqc)))

## column order

columnlist=['sample','job_group','job','fq2urg_gatkid','dup','vm_exp_cnt','vm_cnt','vm_avg_runtime','job_runtime','vm_type','vm_disk','dstat_status','dstat_msg','dstat_log']
job_fastqc=job_fastqc[columnlist]

The number of rows : 288
The number of duplicated jobs : 0
The number of rows dropped duplications : 287


### Text2table for Fastqc

In [354]:
query="MATCH (j:Job:BamFastqc)-[:OUTPUT]->()-[:INPUT_TO]->(t:Job:TextToTable)-[:STATUS]->(d:Dstat) RETURN j.sample as sample, j.name as fq2urg_gatkid, t.name as job, t.duplicate as dup, t.machineType as vm_type, t.durationMinutes as job_runtime, t.diskSize as vm_disk, d.status as dstat_status, d.statusMessage as dstat_msg, d.logging as dstat_log"
job_fastqc_t2t=graph.run(query).to_data_frame()

job_fastqc_t2t['job_group']='QC'
job_fastqc_t2t['vm_exp_cnt']=1
job_fastqc_t2t['vm_cnt']=1
job_fastqc_t2t['vm_avg_runtime']=job_fastqc_t2t['job_runtime']

job_fastqc_t2t=job_fastqc_t2t[columnlist]
job_fastqc_t2t.set_index('sample')

## duplication check

print('The number of rows : ' + str(len(job_fastqc_t2t)))
print('The number of duplicated jobs : ' + str(len(job_fastqc_t2t.loc[job_fastqc_t2t['dup']==True,:])))

## drop duplication

job_fastqc_t2t.drop_duplicates(columnlist,keep='first',inplace=True)
print('The number of rows dropped duplications : ' + str(len(job_fastqc_t2t)))

## column order

columnlist=['sample','job_group','job','fq2urg_gatkid','dup','vm_exp_cnt','vm_cnt','vm_avg_runtime','job_runtime','vm_type','vm_disk','dstat_status','dstat_msg','dstat_log']
job_fastqc_t2t=job_fastqc_t2t[columnlist]

The number of rows : 286
The number of duplicated jobs : 0
The number of rows dropped duplications : 285


In [355]:
#job_fastqc_t2t.loc[job_fastqc_t2t['sample']=='SHIP5119485','dstat_log'][196]
#job_fastqc_t2t['sample'].value_counts()

## Missing 
set(job_fastqc['sample'])-set(job_fastqc_t2t['sample'])

{'SHIP5119489', 'SHIP5141877'}

### Flagstat

In [356]:
query="MATCH (s:Sample), (j:Job:Flagstat)-[:STATUS]->(d:Dstat) WHERE s.sample=j.sample RETURN s.sample as sample, j.name as job, j.duplicate as dup, j.machineType as vm_type, j.durationMinutes as job_runtime, j.diskSize as vm_disk, d.status as dstat_status, d.statusMessage as dstat_msg, d.logging as dstat_log"
job_flagstat = graph.run(query).to_data_frame()

job_flagstat['job_group']='QC'
job_flagstat['fq2urg_gatkid']=None
job_flagstat['vm_exp_cnt']=1
job_flagstat['vm_cnt']=1
job_flagstat['vm_avg_runtime']=job_flagstat['job_runtime']

job_flagstat=job_flagstat[columnlist]
job_flagstat.set_index('sample')

print('The number of rows : ' + str(len(job_flagstat)))
print('The number of duplicated jobs : ' + str(len(job_flagstat.loc[job_flagstat['dup']==True,:])))

## drop duplication

job_flagstat.drop_duplicates(columnlist,keep='first',inplace=True)
print('The number of rows dropped duplications : ' + str(len(job_flagstat)))

## column order

columnlist=['sample','job_group','job','fq2urg_gatkid','dup','vm_exp_cnt','vm_cnt','vm_avg_runtime','job_runtime','vm_type','vm_disk','dstat_status','dstat_msg','dstat_log']
job_flagstat=job_flagstat[columnlist]

The number of rows : 288
The number of duplicated jobs : 1
The number of rows dropped duplications : 288


In [357]:
#job_flagstat.loc[job_flagstat['sample']=='SHIP5119453',:]
#job_flagstat['sample'].value_counts()

## Missing 
#set(job_fastqc['sample'])-set(job_fastqc_t2t['sample'])

### text2table for Flagstat

In [358]:
query="MATCH (s:Sample), (j:Job:Flagstat)-[:OUTPUT]->()-[:INPUT_TO]->(t:Job:TextToTable)-[:STATUS]->(d:Dstat) WHERE s.sample=j.sample RETURN s.sample as sample, j.name as fq2urg_gatkid, t.name as job, t.duplicate as dup, t.machineType as vm_type, t.durationMinutes as job_runtime, t.diskSize as vm_disk, d.status as dstat_status, d.statusMessage as dstat_msg, d.logging as dstat_log"
job_flagstat_t2t=graph.run(query).to_data_frame()

job_flagstat_t2t['job_group']='QC'
job_flagstat_t2t['vm_exp_cnt']=1
job_flagstat_t2t['vm_cnt']=1
job_flagstat_t2t['vm_avg_runtime']=job_flagstat_t2t['job_runtime']

job_flagstat_t2t=job_flagstat_t2t[columnlist]
job_flagstat_t2t.set_index('sample')

print('The number of rows : ' + str(len(job_flagstat_t2t)))
print('The number of duplicated jobs : ' + str(len(job_flagstat_t2t.loc[job_flagstat_t2t['dup']==True,:])))

## drop duplication

job_flagstat_t2t.drop_duplicates(columnlist,keep='first',inplace=True)
print('The number of rows dropped duplications : ' + str(len(job_flagstat_t2t)))

columnlist=['sample','job_group','job','fq2urg_gatkid','dup','vm_exp_cnt','vm_cnt','vm_avg_runtime','job_runtime','vm_type','vm_disk','dstat_status','dstat_msg','dstat_log']
job_flagstat_t2t=job_flagstat_t2t[columnlist]

The number of rows : 287
The number of duplicated jobs : 0
The number of rows dropped duplications : 287


In [359]:
#job_flagstat_t2t.loc[job_flagstat_t2t['sample']=='SHIP5119453',:]
#job_flagstat_t2t['sample'].value_counts()

## Missing 
set(job_flagstat['sample'])-set(job_flagstat_t2t['sample'])

set()

### Vcfstats

In [360]:
query="MATCH (s:Sample), (j:Job:Vcfstats)-[:STATUS]->(d:Dstat) WHERE s.sample=j.sample RETURN s.sample as sample, j.name as job, j.duplicate as dup, j.machineType as vm_type, j.durationMinutes as job_runtime, j.diskSize as vm_disk, d.status as dstat_status, d.statusMessage as dstat_msg, d.logging as dstat_log"
job_vcfstats = graph.run(query).to_data_frame()

job_vcfstats['job_group']='QC'
job_vcfstats['fq2urg_gatkid']=None
job_vcfstats['vm_exp_cnt']=1
job_vcfstats['vm_cnt']=1
job_vcfstats['vm_avg_runtime']=job_vcfstats['job_runtime']

job_vcfstats=job_vcfstats[columnlist]
job_vcfstats.set_index('sample')

print('The number of rows : ' + str(len(job_vcfstats)))
print('The number of duplicated jobs : ' + str(len(job_vcfstats.loc[job_vcfstats['dup']==True,:])))

## drop duplication

job_vcfstats.drop_duplicates(columnlist,keep='first',inplace=True)
print('The number of rows dropped duplications : ' + str(len(job_vcfstats)))

columnlist=['sample','job_group','job','fq2urg_gatkid','dup','vm_exp_cnt','vm_cnt','vm_avg_runtime','job_runtime','vm_type','vm_disk','dstat_status','dstat_msg','dstat_log']
job_vcfstats=job_vcfstats[columnlist]

The number of rows : 287
The number of duplicated jobs : 0
The number of rows dropped duplications : 287


### text2table for Vcfstats

In [361]:
query="MATCH (s:Sample), (j:Job:Vcfstats)-[:OUTPUT]->()-[:INPUT_TO]->(t:Job:TextToTable)-[:STATUS]->(d:Dstat) WHERE s.sample=j.sample RETURN s.sample as sample, j.name as fq2urg_gatkid, t.name as job, t.duplicate as dup, t.machineType as vm_type, t.durationMinutes as job_runtime, t.diskSize as vm_disk, d.status as dstat_status, d.statusMessage as dstat_msg, d.logging as dstat_log"
job_vcfstats_t2t=graph.run(query).to_data_frame()

job_vcfstats_t2t['job_group']='QC'
job_vcfstats_t2t['vm_exp_cnt']=1
job_vcfstats_t2t['vm_cnt']=1
job_vcfstats_t2t['vm_avg_runtime']=job_vcfstats_t2t['job_runtime']

job_vcfstats_t2t=job_vcfstats_t2t[columnlist]
job_vcfstats_t2t.set_index('sample')

print('The number of rows : ' + str(len(job_vcfstats_t2t)))
print('The number of duplicated jobs : ' + str(len(job_vcfstats_t2t.loc[job_vcfstats_t2t['dup']==True,:])))

## drop duplication

job_vcfstats_t2t.drop_duplicates(columnlist,keep='first',inplace=True)
print('The number of rows dropped duplications : ' + str(len(job_vcfstats_t2t)))

columnlist=['sample','job_group','job','fq2urg_gatkid','dup','vm_exp_cnt','vm_cnt','vm_avg_runtime','job_runtime','vm_type','vm_disk','dstat_status','dstat_msg','dstat_log']
job_vcfstats_t2t=job_vcfstats_t2t[columnlist]

The number of rows : 290
The number of duplicated jobs : 0
The number of rows dropped duplications : 287


In [362]:
#job_flagstat_t2t.loc[job_flagstat_t2t['sample']=='SHIP5119453',:]
#job_vcfstats_t2t['sample'].value_counts()

## Missing 
set(job_vcfstats['sample'])-set(job_vcfstats_t2t['sample'])

set()

-------
## Merge 

In [363]:
job_fq2u.head(1)

,sample,job_group,job,fq2urg_gatkid,dup,vm_exp_cnt,vm_cnt,vm_avg_runtime,job_runtime,vm_type,vm_disk,dstat_status,dstat_msg,dstat_log
0,SHIP4948762,GATK,FQ2U,1,None,1,1,58,58,custom-2-7680,None,SUCCESS,Success,gs://gbsc-gcp-project-mvp-test-from-personalis...


In [364]:
job_gatk.head(1)

,sample,job_group,job,fq2urg_gatkid,dup,vm_exp_cnt,vm_cnt,vm_avg_runtime,job_runtime,vm_type,vm_disk,dstat_status,dstat_msg,dstat_log
0,SHIP4948531,GATK,cromwell,e2bbce13-cc8d-4c53-bb02-93d52abb4785,None,1,1,1278,1278,custom-2-12288,None,SUCCESS,Success,gs://gbsc-gcp-project-mvp-test-from-personalis...


In [365]:
job_gatk_stepm.head(1)

,sample,job_group,job,fq2urg_gatkid,dup,vm_exp_cnt,vm_cnt,vm_avg_runtime,job_runtime,vm_type,vm_disk,dstat_status,dstat_msg,dstat_log
0,SHIP4948531,GATK,getbwaversion,e2bbce13-cc8d-4c53-bb02-93d52abb4785,None,1.00,1,3.00,3.57,custom-1-1024,None,SUCCESS,Success,gs://gbsc-gcp-project-mvp-test-from-personalis...


In [376]:
job_df=pd.concat([job_fq2u,job_gatk,job_gatk_stepm,job_fastqc,job_fastqc_t2t,job_flagstat,job_flagstat_t2t,job_vcfstats,job_vcfstats_t2t]).sort_values(['sample','job_group','job','fq2urg_gatkid'])
display(job_df)

,sample,job_group,job,fq2urg_gatkid,dup,vm_exp_cnt,vm_cnt,vm_avg_runtime,job_runtime,vm_type,vm_disk,dstat_status,dstat_msg,dstat_log
309,SHIP4946367,GATK,FQ2U,0,None,1.00000000,1,55.00000000,55.00000000,custom-2-7680,None,SUCCESS,Success,gs://gbsc-gcp-project-mvp-test-from-personalis...
246,SHIP4946367,GATK,FQ2U,1,None,1.00000000,1,56.00000000,56.00000000,custom-2-7680,None,SUCCESS,Success,gs://gbsc-gcp-project-mvp-test-from-personalis...
231,SHIP4946367,GATK,FQ2U,2,None,1.00000000,1,55.00000000,55.00000000,custom-2-7680,None,SUCCESS,Success,gs://gbsc-gcp-project-mvp-test-from-personalis...
218,SHIP4946367,GATK,FQ2U,3,None,1.00000000,1,55.00000000,55.00000000,custom-2-7680,None,SUCCESS,Success,gs://gbsc-gcp-project-mvp-test-from-personalis...
2918,SHIP4946367,GATK,applybqsr,450dc917-48cb-42fe-875e-ca1ca937eabf,None,19.00000000,20,27.68518519,51.43791666,custom-1-3584,None,SUCCESS,Success,gs://gbsc-gcp-project-mvp-test-from-personalis...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
266,SHIP5141934,QC,flagstat,None,None,1.00000000,1,31.00000000,31.00000000,custom-1-3840,None,SUCCESS,Success,gs://gbsc-gcp-project-mvp-test-from-personalis...
264,SHIP5141934,QC,text-to-table,bam-fastqc,None,1.00000000,1,2.00000000,2.00000000,custom-1-3840,None,SUCCESS,Success,gs://gbsc-gcp-project-mvp-test-from-personalis...
265,SHIP5141934,QC,text-to-table,flagstat,None,1.00000000,1,2.00000000,2.00000000,custom-1-3840,None,SUCCESS,Success,gs://gbsc-gcp-project-mvp-test-from-personalis...
258,SHIP5141934,QC,text-to-table,vcfstats,None,1.00000000,1,2.00000000,2.00000000,custom-1-3840,None,SUCCESS,Success,gs://gbsc-gcp-project-mvp-test-from-personalis...


-------
## Cost

### vm_avg_cost

In [377]:
## Custom CPU cost : 0.033174/CPU/Hour, # Custom Mem cost : 0.004446/GB/Hour, # Custom Disk cost : ???
cpu_sd_cost = 0.033174
memg_sd_cost = 0.004446

cpu_pem_cost = 0.00698
memg_pem_cost = 0.00094

## Extract cpu and mem info.
#temp=job_df.loc[:,['job','vm_type']]
job_df.loc[:,'vm_std']=None
job_df.loc[:,'vm_cpu']=0
job_df.loc[:,'vm_mem']=0
job_df[['vm_std','vm_cpu','vm_mem']]=[i.split('-') for i in job_df['vm_type']]
job_df['vm_cpu']=[int(x) for x in job_df['vm_cpu']]
job_df['vm_mem']=[int(x) for x in job_df['vm_mem']]
#columnlist=['job','vm_type','vm_type','cpu','mem']
#temp=temp[columnlist]

In [385]:
# FQ2U and GATK job unit cost with Standard VM
job_df.loc[(job_df['job_group']=='GATK')&(job_df['job'].isin(['FQ2U','cromwell'])),'vm_avg_cost']= np.array(job_df.loc[(job_df['job_group']=='GATK')&(job_df['job'].isin(['FQ2U','cromwell'])),'vm_cpu'])*cpu_sd_cost/60 + np.array(job_df.loc[(job_df['job_group']=='GATK')&(job_df['job'].isin(['FQ2U','cromwell'])),'vm_mem'])*memg_sd_cost/60/1000

# QC unit cost with Standard VM
job_df.loc[(job_df['job_group']=='QC'),'vm_avg_cost']= np.array(job_df.loc[(job_df['job_group']=='QC'),'vm_cpu'])*cpu_sd_cost/60 + np.array(job_df.loc[(job_df['job_group']=='QC'),'vm_mem'])*memg_sd_cost/60/1000

# GATK sub jobs' unit cost with Preemptible VM
#job_df.loc[(job_df['job_group']=='GATK')&(job_df['job'].isin(['FQ2U','cromwell'])==False),'vm_avg_cost']= np.array(job_df.loc[(job_df['job_group']=='GATK')&(job_df['job'].isin(['FQ2U','cromwell'])==False),'vm_cpu'])*cpu_pem_cost/60 + np.array(job_df.loc[(job_df['job_group']=='GATK')&(job_df['job'].isin(['FQ2U','cromwell'])==False),'vm_mem'])*memg_pem_cost/60/1000 

job_df.loc[(job_df['job_group']=='GATK')&(job_df['job'].isin(['FQ2U','cromwell'])==False),'vm_avg_cost']= \
    np.array(job_df.loc[(job_df['job_group']=='GATK')&(job_df['job'].isin(['FQ2U','cromwell'])==False),'vm_cpu'])*cpu_pem_cost/60 + \
    np.array(job_df.loc[(job_df['job_group']=='GATK')&(job_df['job'].isin(['FQ2U','cromwell'])==False),'vm_mem'])*memg_pem_cost/60/1000

### job cost

In [386]:
pd.set_option('display.float_format', lambda x: '%.8f' % x)

## FQ2U
job_df.loc[job_df['job']=='FQ2U','job_cost']=np.array(job_df.loc[job_df['job']=='FQ2U','job_runtime'])*np.array(job_df.loc[job_df['job']=='FQ2U','vm_avg_cost'])
#merged_cost_df.head(2)

## GATK
job_df.loc[job_df['job']=='cromwell','job_cost']=np.array(job_df.loc[job_df['job']=='cromwell','job_runtime'])*np.array(job_df.loc[job_df['job']=='cromwell','vm_avg_cost'])
#merged_cost_df[merged_cost_df['job']=='GATK'].head(2)

## QC
job_df.loc[job_df['job_group']=='QC','job_cost']=np.array(job_df.loc[job_df['job_group']=='QC','job_runtime'])*np.array(job_df.loc[job_df['job_group']=='QC','vm_avg_cost'])
#merged_cost_df[merged_cost_df['job']=='GATK'].head(2)

## GATK steps
job_df.loc[(job_df['job_group']=='GATK')&(job_df['job']!='FQ2U') & (job_df['job']!='cromwell'),'job_cost']=np.array(job_df.loc[(job_df['job_group']=='GATK')&(job_df['job']!='FQ2U') & (job_df['job']!='cromwell'),'vm_cnt']) \
*np.array(job_df.loc[(job_df['job_group']=='GATK')&(job_df['job']!='FQ2U') & (job_df['job']!='cromwell'),'vm_avg_runtime'])*np.array(job_df.loc[(job_df['job_group']=='GATK')&(job_df['job']!='FQ2U') & (job_df['job']!='cromwell'),'vm_avg_cost'])

In [387]:
columnlist=['sample','job_group','job','fq2urg_gatkid','dup','vm_exp_cnt','vm_cnt','vm_avg_runtime','job_runtime','vm_type','vm_cpu','vm_mem','vm_disk','vm_avg_cost','job_cost','dstat_status','dstat_msg','dstat_log']
job_df=job_df[columnlist]
job_df.to_csv('job-based-analysis-v058-1.csv',index=False)

### Upload CSV Files to BigQuery

In [ ]:
# table_id='mvp_wgs35_v058_1.job_based_analysis'
# projectid='gbsc-gcp-project-mvp-test'

# pandas_gbq.to_gbq(
#     job_df, table_id, project_id=projectid, if_exists='replace',
# )